All model evaluations use 'gemini-2.5-flash-preview-04-17' model for the LLM evaluation 

Evaluating Baseline qwen-2.5 3B model output







In [ ]:
import json
from typing import Any, Dict, List
import ntpath
import evals
import csv
import xml.etree.ElementTree as ET
import tqdm
import statistics

print("Baseline 3B")
def parse_jsonl(file_path: str) -> List[Dict[str, Any]]:
    """
    Parse a JSONL file where each line is a JSON object containing:
      - image_path: str
      - recipe_xml: str
      - ing_vecs: List[List[float]]
      - step_vecs: List[List[float]]

    Returns a list of dictionaries with those keys.
    """
    entries: List[Dict[str, Any]] = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data = json.loads(line)
            entry = {
                'image_path': 'Food Images/Food Images/' + ntpath.basename(data.get('image_path', '')),
                'recipe_xml': data.get('recipe_xml', ''),
                'ing_vecs': data.get('ing_vecs', []),
                'step_vecs': data.get('step_vecs', []),
            }
            entries.append(entry)
    return entries

data = parse_jsonl('/Users/BenChung/Desktop/CSCI 467/vision-r1/data/dev_stage/dev_pairs_vec.jsonl')
# print(data[0]['image_path'])

results = []
csv_path = '/Users/BenChung/Desktop/CSCI 467/Generated Recipes Baseline 3B.csv'
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        xml_str = row.get("response", "")
        category = None
        if xml_str:
            try:
                root = ET.fromstring(xml_str)
                cat_elem = root.find('.//response_category')
                if cat_elem is not None:
                    category = cat_elem.text
            except ET.ParseError:
                # malformed XML—leave category as None
                pass
        # row['response_category'] = category
        results.append(xml_str)



model_xml_outputs = results

# print(model_xml_outputs)

aggregate_evals = []

# print(data)

for i, example in tqdm.tqdm(enumerate(data)):
    # print(model_xml_outputs[i])
    temp = evals.compute_evals(model_xml_outputs[i], example)
    if temp == None: continue
    else: 
        aggregate_evals.append(temp)
        print("cosine_similarity: ")
        print(temp['cosine_similarity'])
        print("bleu_scores: ")
        print(temp["bleu_scores"])
        print("rouge_scores: ")
        print(temp['rouge_scores'])
        print("llm_evaluation: ")
        print(temp['llm_evaluation'])

cos_steps       = [e['cosine_similarity']['steps']       for e in aggregate_evals]
cos_ing         = [e['cosine_similarity']['ingredients'] for e in aggregate_evals]
bleu_steps      = [e['bleu_scores']['steps']             for e in aggregate_evals]
bleu_ing        = [e['bleu_scores']['ingredients']       for e in aggregate_evals]

rouge_s1_steps  = [e['rouge_scores']['steps']['rouge1']       for e in aggregate_evals]
rougeL_steps    = [e['rouge_scores']['steps']['rougeL']       for e in aggregate_evals]
rouge_s1_ing    = [e['rouge_scores']['ingredients']['rouge1'] for e in aggregate_evals]
rougeL_ing      = [e['rouge_scores']['ingredients']['rougeL'] for e in aggregate_evals]

feas_scores     = [e['llm_evaluation']['feasibility_score']   for e in aggregate_evals]

def stats(vals: List[float]) -> Dict[str, float]:
    return {
        'mean': statistics.mean(vals),
        'std':  statistics.pstdev(vals)
    }

print(aggregate_evals)
print()
print()
print("Final averages and std deviations:")

new_dict = {'cosine_similarity': {
            'steps':       stats(cos_steps),
            'ingredients': stats(cos_ing),
        },
        'bleu_scores': {
            'steps':       stats(bleu_steps),
            'ingredients': stats(bleu_ing)
        },
        'rouge_scores': {
            'steps': {
                'rouge1': stats(rouge_s1_steps),
                'rougeL': stats(rougeL_steps),
            },
            'ingredients': {
                'rouge1': stats(rouge_s1_ing),
                'rougeL': stats(rougeL_ing),
            }
        },
        'llm_feasibility_score': stats(feas_scores)
}
print("cosine_similarity:\n\tsteps: ", end="")
print(new_dict['cosine_similarity']['steps'])
print("\tingredients: ", end="")
print(new_dict['cosine_similarity']['ingredients'])
print("bleu_scores:\n\tsteps: ", end="")
print(new_dict['bleu_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['bleu_scores']['ingredients'])
print("rouge_scores:\n\tsteps: ", end="")
print(new_dict['rouge_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['rouge_scores']['ingredients'])
print("llm_feasibility_score: ", end="")
print(new_dict['llm_feasibility_score'])



Baseline 3B


0it [00:00, ?it/s]

Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Calculating LLM evaluation...


1it [00:12, 12.96s/it]

cosine_similarity: 
{'steps': np.float64(0.5906911656512902), 'ingredients': np.float64(0.5511275781087667)}
bleu_scores: 
{'steps': 1.1970014442835451e-05, 'ingredients': 0.04539285423716415}
rouge_scores: 
{'steps': {'rouge1': 0.10783855384135889, 'rougeL': 0.08991740688795387}, 'ingredients': {'rouge1': 0.4327283827283827, 'rougeL': 0.41487123987123986}}
llm_evaluation: 
{'feasibility_score': np.float64(0.3), 'comment': 'The predicted recipe is a plausible method for making a simple cheesy pasta with tomato sauce, but it significantly deviates from the ground truth recipe and the dish depicted in the image. Crucially, it omits the lentils, which are a central component visible in the photo and included in the ground truth. The cooking method is also too short to develop a rich sauce like a bolognese, whether meat-based or lentil-based. It would not produce the dish shown.', 'full': '{\n  "evaluation": {\n    "feasibility_score": "0.3",\n    "comment": "The predicted recipe is a plau

...

cosine_similarity: 
{'steps': np.float64(0.579536517301945), 'ingredients': np.float64(0.41306274479236027)}
bleu_scores: 
{'steps': 3.8616373748248454e-10, 'ingredients': 0.06247873011072971}
rouge_scores: 
{'steps': {'rouge1': 0.062086895413743036, 'rougeL': 0.0493808374856841}, 'ingredients': {'rouge1': 0.2814814814814815, 'rougeL': 0.2814814814814815}}
llm_evaluation: 
{'feasibility_score': np.float64(0.0), 'comment': 'The predicted recipe is completely misaligned with the image and ground truth. It describes a Simple Caesar Salad, using ingredients like romaine lettuce, croutons, and Parmesan cheese. The image and ground truth clearly show a Thai Beef Salad with steak, butter lettuce, cucumber, tomatoes, onion, herbs, and peanuts, dressed in a chile-lime sauce. The predicted recipe would produce a entirely different dish, making it completely infeasible for creating the salad pictured.', 'full': '{\n "evaluation": {\n  "feasibility_score": "0.0",\n  "comment": "The predicted recip

18it [02:15,  6.46s/it]

cosine_similarity: 
{'steps': np.float64(0.5613000236520391), 'ingredients': np.float64(0.5664145607110855)}
bleu_scores: 
{'steps': 2.726652657747824e-07, 'ingredients': 0.05211236971927989}
rouge_scores: 
{'steps': {'rouge1': 0.06558047464878089, 'rougeL': 0.050570620631680205}, 'ingredients': {'rouge1': 0.4028860028860029, 'rougeL': 0.4028860028860029}}
llm_evaluation: 
{'feasibility_score': np.float64(0.3), 'comment': "The predicted recipe for Bacon and Tomato Salad does not realistically produce the dish shown in the image. While it includes bacon and tomatoes, it completely misses the lettuce, the creamy horseradish mayonnaise sauce, and the crispy breadcrumbs (or nuts) clearly visible. The predicted recipe's dressing is a vinaigrette with honey, and it adds feta cheese, which are not present in the visual or the ground truth. Furthermore, the image shows components laid out for assembly, whereas the predicted recipe describes a simple mixed salad.", 'full': '{\n "evaluation": {\

20it [02:23,  7.20s/it]

cosine_similarity: 
{'steps': np.float64(0.5014523289865175), 'ingredients': np.float64(0.5461490578487844)}
bleu_scores: 
{'steps': 0.0025732887108971805, 'ingredients': 0.06466914919557828}
rouge_scores: 
{'steps': {'rouge1': 0.16875404239745323, 'rougeL': 0.12632188368622477}, 'ingredients': {'rouge1': 0.42875679875679873, 'rougeL': 0.42875679875679873}}
llm_evaluation: 
{'feasibility_score': np.float64(0.2), 'comment': "The predicted recipe is feasible as a standalone recipe, but it completely fails to match the image or the ground-truth recipe. The image and ground truth feature green and wax beans, while the prediction uses white beans, fundamentally altering the dish's appearance and texture. It also uses a different mix of herbs and a lemon-honey dressing instead of the vinegar-based one with shallots and basil specified in the ground truth. Although the steps are logical for the ingredients listed, the core ingredient discrepancy makes it irrelevant to the intended dish.", 'fu

Evaluating Baseline qwen-2.5 7B model output







In [4]:
import json
from typing import Any, Dict, List
import ntpath
import evals
import csv
import xml.etree.ElementTree as ET
import tqdm
import statistics

print("Baseline 3B")
def parse_jsonl(file_path: str) -> List[Dict[str, Any]]:
    """
    Parse a JSONL file where each line is a JSON object containing:
      - image_path: str
      - recipe_xml: str
      - ing_vecs: List[List[float]]
      - step_vecs: List[List[float]]

    Returns a list of dictionaries with those keys.
    """
    entries: List[Dict[str, Any]] = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data = json.loads(line)
            entry = {
                'image_path': 'Food Images/Food Images/' + ntpath.basename(data.get('image_path', '')),
                'recipe_xml': data.get('recipe_xml', ''),
                'ing_vecs': data.get('ing_vecs', []),
                'step_vecs': data.get('step_vecs', []),
            }
            entries.append(entry)
    return entries

data = parse_jsonl('/Users/BenChung/Desktop/CSCI 467/vision-r1/data/dev_stage/dev_pairs_vec.jsonl')
# print(data[0]['image_path'])

results = []
csv_path = '/Users/BenChung/Desktop/CSCI 467/Generated Recipes Baseline 7B.csv'
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        xml_str = row.get("response", "")
        category = None
        if xml_str:
            try:
                root = ET.fromstring(xml_str)
                cat_elem = root.find('.//response_category')
                if cat_elem is not None:
                    category = cat_elem.text
            except ET.ParseError:
                # malformed XML—leave category as None
                pass
        # row['response_category'] = category
        results.append(xml_str)



model_xml_outputs = results

# print(model_xml_outputs)

aggregate_evals = []

# print(data)

for i, example in tqdm.tqdm(enumerate(data)):
    # print(model_xml_outputs[i])
    temp = evals.compute_evals(model_xml_outputs[i], example)
    if temp == None: continue
    else: 
        aggregate_evals.append(temp)
        print("cosine_similarity: ")
        print(temp['cosine_similarity'])
        print("bleu_scores: ")
        print(temp["bleu_scores"])
        print("rouge_scores: ")
        print(temp['rouge_scores'])
        print("llm_evaluation: ")
        print(temp['llm_evaluation'])

cos_steps       = [e['cosine_similarity']['steps']       for e in aggregate_evals]
cos_ing         = [e['cosine_similarity']['ingredients'] for e in aggregate_evals]
bleu_steps      = [e['bleu_scores']['steps']             for e in aggregate_evals]
bleu_ing        = [e['bleu_scores']['ingredients']       for e in aggregate_evals]

rouge_s1_steps  = [e['rouge_scores']['steps']['rouge1']       for e in aggregate_evals]
rougeL_steps    = [e['rouge_scores']['steps']['rougeL']       for e in aggregate_evals]
rouge_s1_ing    = [e['rouge_scores']['ingredients']['rouge1'] for e in aggregate_evals]
rougeL_ing      = [e['rouge_scores']['ingredients']['rougeL'] for e in aggregate_evals]

feas_scores     = [e['llm_evaluation']['feasibility_score']   for e in aggregate_evals]

def stats(vals: List[float]) -> Dict[str, float]:
    return {
        'mean': statistics.mean(vals),
        'std':  statistics.pstdev(vals)
    }

print(aggregate_evals)
print()
print()
print("Final averages and std deviations:")

new_dict = {'cosine_similarity': {
            'steps':       stats(cos_steps),
            'ingredients': stats(cos_ing),
        },
        'bleu_scores': {
            'steps':       stats(bleu_steps),
            'ingredients': stats(bleu_ing)
        },
        'rouge_scores': {
            'steps': {
                'rouge1': stats(rouge_s1_steps),
                'rougeL': stats(rougeL_steps),
            },
            'ingredients': {
                'rouge1': stats(rouge_s1_ing),
                'rougeL': stats(rougeL_ing),
            }
        },
        'llm_feasibility_score': stats(feas_scores)
}
print("cosine_similarity:\n\tsteps: ", end="")
print(new_dict['cosine_similarity']['steps'])
print("\tingredients: ", end="")
print(new_dict['cosine_similarity']['ingredients'])
print("bleu_scores:\n\tsteps: ", end="")
print(new_dict['bleu_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['bleu_scores']['ingredients'])
print("rouge_scores:\n\tsteps: ", end="")
print(new_dict['rouge_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['rouge_scores']['ingredients'])
print("llm_feasibility_score: ", end="")
print(new_dict['llm_feasibility_score'])



Baseline 3B


0it [00:00, ?it/s]

Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Calculating LLM evaluation...


1it [00:09,  9.70s/it]

cosine_similarity: 
{'steps': np.float64(0.5041167389064803), 'ingredients': np.float64(0.6091995971775844)}
bleu_scores: 
{'steps': 1.764921602125214e-05, 'ingredients': 0.06920452110926026}
rouge_scores: 
{'steps': {'rouge1': 0.10603896705918883, 'rougeL': 0.08000717791625267}, 'ingredients': {'rouge1': 0.4761327561327562, 'rougeL': 0.4332756132756133}}
llm_evaluation: 
{'feasibility_score': np.float64(0.5), 'comment': 'The predicted recipe is a feasible set of instructions for making a standard Macaroni with Meat Sauce. However, it significantly deviates from the ground-truth recipe which is a Lentil Bolognese, using lentils instead of meat. While the visual result of a chunky sauce with cheese could be similar, the predicted recipe specifies elbow macaroni, which differs from the shaped pasta shown and suggested in the ground truth. The method is practical, but the core ingredients and specific pasta shape are mismatched compared to the provided context.', 'full': '{\n  "evaluation

...

cosine_similarity: 
{'steps': np.float64(0.5162384494284241), 'ingredients': np.float64(0.5716005077553936)}
bleu_scores: 
{'steps': 2.3083644219448536e-06, 'ingredients': 0.06326664735685866}
rouge_scores: 
{'steps': {'rouge1': 0.06920681415797984, 'rougeL': 0.04950435076024543}, 'ingredients': {'rouge1': 0.4258494031221304, 'rougeL': 0.4258494031221304}}
llm_evaluation: 
{'feasibility_score': np.float64(0.1), 'comment': 'The predicted recipe is entirely unrelated to the dish shown in the image and the ground-truth recipe. It describes a grilled steak salad with different ingredients (steak, spinach, feta, vinaigrette) and a tossed preparation method, which does not match the deconstructed layout and components visible in the image (bacon strips, lettuce leaves, creamy sauce, breadcrumbs/nuts). While the predicted recipe is technically feasible as a standalone dish, it completely fails to replicate or align with the provided context.', 'full': '{\n "evaluation": {\n  "feasibility_scor

19it [02:21,  7.77s/it]

cosine_similarity: 
{'steps': np.float64(0.7282422788137265), 'ingredients': np.float64(0.6427167822005774)}
bleu_scores: 
{'steps': 6.611363028745495e-05, 'ingredients': 0.06442929602381638}
rouge_scores: 
{'steps': {'rouge1': 0.13606696011482647, 'rougeL': 0.09194080058151566}, 'ingredients': {'rouge1': 0.42954267954267955, 'rougeL': 0.4035686535686536}}
llm_evaluation: 
{'feasibility_score': np.float64(0.65), 'comment': "The predicted recipe is feasible for grilling a steak with herb butter, but it doesn't fully capture the specific method and visual result shown in the image and described in the ground truth. The ground truth uses a 'board dressing' applied to the sliced steak, coating the pieces, whereas the predicted recipe suggests serving the herb butter on the side. The specified grilling time in the predicted recipe also seems short for a thick steak often used in such preparations, potentially not achieving the char or doneness seen in the image. While functional, it's not a

20it [02:29,  7.46s/it]

cosine_similarity: 
{'steps': np.float64(0.5483549516360793), 'ingredients': np.float64(0.621614139660321)}
bleu_scores: 
{'steps': 0.002667752782945187, 'ingredients': 0.06663059344033823}
rouge_scores: 
{'steps': {'rouge1': 0.19132738792690057, 'rougeL': 0.15471997823475628}, 'ingredients': {'rouge1': 0.5056203056203056, 'rougeL': 0.5056203056203056}}
llm_evaluation: 
{'feasibility_score': np.float64(0.3), 'comment': "This predicted recipe deviates significantly from the ground-truth recipe and the image. It omits key ingredients like wax beans and shallots, which are visible and listed in the original. It also adds ingredients like cherry tomatoes, parsley, and Dijon mustard, which are not present. The instruction for preparing the beans is vague and doesn't align with properly cooking fresh beans as described in the ground truth. While it outlines making a salad with a dressing, it would not produce the specific dish shown in the image or described by the original recipe due to the

Evaluating Baseline qwen-2.5 32B model output







In [5]:
import json
from typing import Any, Dict, List
import ntpath
import evals
import csv
import xml.etree.ElementTree as ET
import tqdm
import statistics

print("Baseline 3B")
def parse_jsonl(file_path: str) -> List[Dict[str, Any]]:
    """
    Parse a JSONL file where each line is a JSON object containing:
      - image_path: str
      - recipe_xml: str
      - ing_vecs: List[List[float]]
      - step_vecs: List[List[float]]

    Returns a list of dictionaries with those keys.
    """
    entries: List[Dict[str, Any]] = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data = json.loads(line)
            entry = {
                'image_path': 'Food Images/Food Images/' + ntpath.basename(data.get('image_path', '')),
                'recipe_xml': data.get('recipe_xml', ''),
                'ing_vecs': data.get('ing_vecs', []),
                'step_vecs': data.get('step_vecs', []),
            }
            entries.append(entry)
    return entries

data = parse_jsonl('/Users/BenChung/Desktop/CSCI 467/vision-r1/data/dev_stage/dev_pairs_vec.jsonl')
# print(data[0]['image_path'])

results = []
csv_path = '/Users/BenChung/Desktop/CSCI 467/Generated Recipes Baseline 32B.csv'
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        xml_str = row.get("response", "")
        category = None
        if xml_str:
            try:
                root = ET.fromstring(xml_str)
                cat_elem = root.find('.//response_category')
                if cat_elem is not None:
                    category = cat_elem.text
            except ET.ParseError:
                # malformed XML—leave category as None
                pass
        # row['response_category'] = category
        results.append(xml_str)



model_xml_outputs = results

# print(model_xml_outputs)

aggregate_evals = []

# print(data)

for i, example in tqdm.tqdm(enumerate(data)):
    # print(model_xml_outputs[i])
    temp = evals.compute_evals(model_xml_outputs[i], example)
    if temp == None: continue
    else: 
        aggregate_evals.append(temp)
        print("cosine_similarity: ")
        print(temp['cosine_similarity'])
        print("bleu_scores: ")
        print(temp["bleu_scores"])
        print("rouge_scores: ")
        print(temp['rouge_scores'])
        print("llm_evaluation: ")
        print(temp['llm_evaluation'])

cos_steps       = [e['cosine_similarity']['steps']       for e in aggregate_evals]
cos_ing         = [e['cosine_similarity']['ingredients'] for e in aggregate_evals]
bleu_steps      = [e['bleu_scores']['steps']             for e in aggregate_evals]
bleu_ing        = [e['bleu_scores']['ingredients']       for e in aggregate_evals]

rouge_s1_steps  = [e['rouge_scores']['steps']['rouge1']       for e in aggregate_evals]
rougeL_steps    = [e['rouge_scores']['steps']['rougeL']       for e in aggregate_evals]
rouge_s1_ing    = [e['rouge_scores']['ingredients']['rouge1'] for e in aggregate_evals]
rougeL_ing      = [e['rouge_scores']['ingredients']['rougeL'] for e in aggregate_evals]

feas_scores     = [e['llm_evaluation']['feasibility_score']   for e in aggregate_evals]

def stats(vals: List[float]) -> Dict[str, float]:
    return {
        'mean': statistics.mean(vals),
        'std':  statistics.pstdev(vals)
    }

print(aggregate_evals)
print()
print()
print("Final averages and std deviations:")

new_dict = {'cosine_similarity': {
            'steps':       stats(cos_steps),
            'ingredients': stats(cos_ing),
        },
        'bleu_scores': {
            'steps':       stats(bleu_steps),
            'ingredients': stats(bleu_ing)
        },
        'rouge_scores': {
            'steps': {
                'rouge1': stats(rouge_s1_steps),
                'rougeL': stats(rougeL_steps),
            },
            'ingredients': {
                'rouge1': stats(rouge_s1_ing),
                'rougeL': stats(rougeL_ing),
            }
        },
        'llm_feasibility_score': stats(feas_scores)
}
print("cosine_similarity:\n\tsteps: ", end="")
print(new_dict['cosine_similarity']['steps'])
print("\tingredients: ", end="")
print(new_dict['cosine_similarity']['ingredients'])
print("bleu_scores:\n\tsteps: ", end="")
print(new_dict['bleu_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['bleu_scores']['ingredients'])
print("rouge_scores:\n\tsteps: ", end="")
print(new_dict['rouge_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['rouge_scores']['ingredients'])
print("llm_feasibility_score: ", end="")
print(new_dict['llm_feasibility_score'])



Baseline 3B


0it [00:00, ?it/s]

Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Calculating LLM evaluation...


1it [00:06,  6.84s/it]

cosine_similarity: 
{'steps': np.float64(0.4318893307474136), 'ingredients': np.float64(0.4781369234449445)}
bleu_scores: 
{'steps': 0.005069595493544869, 'ingredients': 0.051222312243778906}
rouge_scores: 
{'steps': {'rouge1': 0.17066815247422207, 'rougeL': 0.12826228792179942}, 'ingredients': {'rouge1': 0.3316738816738817, 'rougeL': 0.3316738816738817}}
llm_evaluation: 
{'feasibility_score': np.float64(0.1), 'comment': "The predicted recipe for 'Tuna Mac and Cheese' is completely different from the dish shown in the image, which is a lentil bolognese. The ingredients (tuna, mozzarella, creamy sauce base) and preparation steps (making a cheese sauce, baking) do not align with the visual appearance (tomato/lentil sauce, grated cheese topping) or the ground truth recipe. The pasta shape is also incorrect. This recipe would produce a wholly different dish.", 'full': '{\n  "evaluation": {\n    "feasibility_score": "0.1",\n    "comment": "The predicted recipe for \'Tuna Mac and Cheese\' is

...

cosine_similarity: 
{'steps': np.float64(0.5161489570079049), 'ingredients': np.float64(0.5359677148119334)}
bleu_scores: 
{'steps': 4.2303095449574187e-05, 'ingredients': 0.05488369506220401}
rouge_scores: 
{'steps': {'rouge1': 0.0912479362107331, 'rougeL': 0.05956939355578374}, 'ingredients': {'rouge1': 0.37811688311688313, 'rougeL': 0.37811688311688313}}
llm_evaluation: 
{'feasibility_score': np.float64(0.2), 'comment': "The predicted recipe is completely different from the dish shown and described in the ground truth. It misidentifies the main protein as prosciutto instead of bacon and proposes a completely different dressing (Lemon-Dijon vinaigrette vs. creamy horseradish mayonnaise). Key components visible in the image and listed in the ground truth, such as breadcrumbs/croutons and basil, are also missing. While the predicted recipe is a feasible dish in itself, it does not match the image or the ground-truth recipe's ingredients or steps.", 'full': '{\n "evaluation": {\n  "feas

19it [02:30,  8.40s/it]

cosine_similarity: 
{'steps': np.float64(0.5797421530664408), 'ingredients': np.float64(0.5685595175090796)}
bleu_scores: 
{'steps': 0.00045424455120924776, 'ingredients': 0.05271190877675713}
rouge_scores: 
{'steps': {'rouge1': 0.12043325782776303, 'rougeL': 0.07708517339341306}, 'ingredients': {'rouge1': 0.3325876504447933, 'rougeL': 0.3325876504447933}}
llm_evaluation: 
{'feasibility_score': np.float64(0.4), 'comment': "The predicted recipe includes grilling steak with herb butter, which aligns somewhat with the image's sliced steak topped with herbs. However, it deviates significantly by adding seared vegetables and a balsamic dressing not present in the visual or ground truth. The ground truth recipe and image feature a T-bone steak specifically and use a 'board dressing' method which is key to the appearance of the sliced steak. The predicted recipe misses these crucial details, making it only moderately feasible to produce the exact dish pictured.", 'full': '{\n "evaluation": {\

20it [02:36,  7.85s/it]

cosine_similarity: 
{'steps': np.float64(0.45894492026350947), 'ingredients': np.float64(0.5675192502668893)}
bleu_scores: 
{'steps': 0.009178556265743926, 'ingredients': 0.068566864871851}
rouge_scores: 
{'steps': {'rouge1': 0.22866708385481851, 'rougeL': 0.16438429981921845}, 'ingredients': {'rouge1': 0.4013708513708514, 'rougeL': 0.4013708513708514}}
llm_evaluation: 
{'feasibility_score': np.float64(0.1), 'comment': "The predicted recipe shares only the title with the ground truth and image. The ingredients listed (lettuce, butter/kidney beans, stilton cheese, almonds) are completely different from the string beans, shallots, and basil in the ground truth recipe and pictured. Consequently, the cooking steps and the resulting dish are entirely different from what is shown in the image. While it describes a plausible recipe for *a* salad, it does not align with the specific 'Herbed Bean Salad' depicted, hence the very low feasibility score.", 'full': '{\n  "evaluation": {\n    "feasib

Evaluating GRPO fine-tuned qwen-2.5 3B model output







In [6]:
import json
from typing import Any, Dict, List
import ntpath
import evals
import csv
import xml.etree.ElementTree as ET
import tqdm
import statistics

print("Baseline 3B")
def parse_jsonl(file_path: str) -> List[Dict[str, Any]]:
    """
    Parse a JSONL file where each line is a JSON object containing:
      - image_path: str
      - recipe_xml: str
      - ing_vecs: List[List[float]]
      - step_vecs: List[List[float]]

    Returns a list of dictionaries with those keys.
    """
    entries: List[Dict[str, Any]] = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data = json.loads(line)
            entry = {
                'image_path': 'Food Images/Food Images/' + ntpath.basename(data.get('image_path', '')),
                'recipe_xml': data.get('recipe_xml', ''),
                'ing_vecs': data.get('ing_vecs', []),
                'step_vecs': data.get('step_vecs', []),
            }
            entries.append(entry)
    return entries

data = parse_jsonl('/Users/BenChung/Desktop/CSCI 467/vision-r1/data/dev_stage/dev_pairs_vec.jsonl')
# print(data[0]['image_path'])

results = []
csv_path = '/Users/BenChung/Desktop/CSCI 467/Generated Recipes.csv'
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        xml_str = row.get("response", "")
        category = None
        if xml_str:
            try:
                root = ET.fromstring(xml_str)
                cat_elem = root.find('.//response_category')
                if cat_elem is not None:
                    category = cat_elem.text
            except ET.ParseError:
                # malformed XML—leave category as None
                pass
        # row['response_category'] = category
        results.append(xml_str)



model_xml_outputs = results

# print(model_xml_outputs)

aggregate_evals = []

# print(data)

for i, example in tqdm.tqdm(enumerate(data)):
    # print(model_xml_outputs[i])
    temp = evals.compute_evals(model_xml_outputs[i], example)
    if temp == None: continue
    else: 
        aggregate_evals.append(temp)
        print("cosine_similarity: ")
        print(temp['cosine_similarity'])
        print("bleu_scores: ")
        print(temp["bleu_scores"])
        print("rouge_scores: ")
        print(temp['rouge_scores'])
        print("llm_evaluation: ")
        print(temp['llm_evaluation'])

cos_steps       = [e['cosine_similarity']['steps']       for e in aggregate_evals]
cos_ing         = [e['cosine_similarity']['ingredients'] for e in aggregate_evals]
bleu_steps      = [e['bleu_scores']['steps']             for e in aggregate_evals]
bleu_ing        = [e['bleu_scores']['ingredients']       for e in aggregate_evals]

rouge_s1_steps  = [e['rouge_scores']['steps']['rouge1']       for e in aggregate_evals]
rougeL_steps    = [e['rouge_scores']['steps']['rougeL']       for e in aggregate_evals]
rouge_s1_ing    = [e['rouge_scores']['ingredients']['rouge1'] for e in aggregate_evals]
rougeL_ing      = [e['rouge_scores']['ingredients']['rougeL'] for e in aggregate_evals]

feas_scores     = [e['llm_evaluation']['feasibility_score']   for e in aggregate_evals]

def stats(vals: List[float]) -> Dict[str, float]:
    return {
        'mean': statistics.mean(vals),
        'std':  statistics.pstdev(vals)
    }

print(aggregate_evals)
print()
print()
print("Final averages and std deviations:")

new_dict = {'cosine_similarity': {
            'steps':       stats(cos_steps),
            'ingredients': stats(cos_ing),
        },
        'bleu_scores': {
            'steps':       stats(bleu_steps),
            'ingredients': stats(bleu_ing)
        },
        'rouge_scores': {
            'steps': {
                'rouge1': stats(rouge_s1_steps),
                'rougeL': stats(rougeL_steps),
            },
            'ingredients': {
                'rouge1': stats(rouge_s1_ing),
                'rougeL': stats(rougeL_ing),
            }
        },
        'llm_feasibility_score': stats(feas_scores)
}
print("cosine_similarity:\n\tsteps: ", end="")
print(new_dict['cosine_similarity']['steps'])
print("\tingredients: ", end="")
print(new_dict['cosine_similarity']['ingredients'])
print("bleu_scores:\n\tsteps: ", end="")
print(new_dict['bleu_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['bleu_scores']['ingredients'])
print("rouge_scores:\n\tsteps: ", end="")
print(new_dict['rouge_scores']['steps'])
print("\tingredients: ", end="")
print(new_dict['rouge_scores']['ingredients'])
print("llm_feasibility_score: ", end="")
print(new_dict['llm_feasibility_score'])



Baseline 3B


0it [00:00, ?it/s]

Calculating cosine similarity...
Calculating BLEU scores...
Calculating ROUGE scores...
Calculating LLM evaluation...


1it [00:07,  7.90s/it]

cosine_similarity: 
{'steps': np.float64(0.5324286358401825), 'ingredients': np.float64(0.6754974909290884)}
bleu_scores: 
{'steps': 0.00012925265683217318, 'ingredients': 0.052170028630432755}
rouge_scores: 
{'steps': {'rouge1': 0.11908838342724229, 'rougeL': 0.09250725954244311}, 'ingredients': {'rouge1': 0.48679306804306804, 'rougeL': 0.45107878232878235}}
llm_evaluation: 
{'feasibility_score': np.float64(0.3), 'comment': "The predicted recipe is fundamentally different from both the image and the ground truth. The image clearly shows a textured sauce with what appear to be small particles (like lentils or ground meat) and a non-creamy base, served with cavatappi or similar pasta shape. The predicted recipe calls for penne pasta and a creamy tomato sauce with cherry tomatoes, which would look completely different. While the steps describe making a pasta dish, the ingredients and method do not match the visual target or the actual recipe. The inclusion of an oven preheat step that is

...

cosine_similarity: 
{'steps': np.float64(0.5997848129246802), 'ingredients': np.float64(0.5954396312319775)}
bleu_scores: 
{'steps': 5.157218964180255e-07, 'ingredients': 0.07598523699474391}
rouge_scores: 
{'steps': {'rouge1': 0.07214367709311893, 'rougeL': 0.06051286269413844}, 'ingredients': {'rouge1': 0.38484848484848483, 'rougeL': 0.38484848484848483}}
llm_evaluation: 
{'feasibility_score': np.float64(0.1), 'comment': "The predicted recipe does not realistically produce the Thai Beef Salad shown in the image or described in the ground truth. While it identifies some basic salad ingredients like greens, tomatoes, and onions, it crucially misses the main protein (beef), cucumber, peanuts, and the specific chili-lime dressing visible. The inclusion of feta cheese is inaccurate for this dish. The instructions are overly simplistic, omitting the necessary steps for cooking the beef and preparing the correct dressing. The model's thinking process correctly identifies basic vegetables an

19it [02:32,  7.70s/it]

cosine_similarity: 
{'steps': np.float64(0.6423547652678897), 'ingredients': np.float64(0.47107454523320774)}
bleu_scores: 
{'steps': 0.0003296566738260276, 'ingredients': 0.019918393983362474}
rouge_scores: 
{'steps': {'rouge1': 0.13137714046284643, 'rougeL': 0.08352328695867307}, 'ingredients': {'rouge1': 0.20105820105820105, 'rougeL': 0.13756613756613756}}
llm_evaluation: 
{'feasibility_score': np.float64(0.35), 'comment': "The predicted recipe correctly identifies that the dish involves grilled steak cooked to medium-rare with a charred exterior. However, it significantly deviates from the ground truth and the visual evidence in the image. It includes roasted potatoes, which are not present. It suggests an 'herb butter' which is a different preparation than the liquid 'board dressing' shown on the sliced steak and described in the ground truth. The crucial technique of slicing the steak directly into the dressing on the board, visible in the image and central to the ground truth, i

20it [02:39,  7.98s/it]

cosine_similarity: 
{'steps': np.float64(0.4502754074629116), 'ingredients': np.float64(0.513716630432621)}
bleu_scores: 
{'steps': 0.0031078871291768937, 'ingredients': 0.054619286899042205}
rouge_scores: 
{'steps': {'rouge1': 0.14322067384947712, 'rougeL': 0.1136034351044493}, 'ingredients': {'rouge1': 0.4933954933954933, 'rougeL': 0.4933954933954933}}
llm_evaluation: 
{'feasibility_score': np.float64(0.1), 'comment': "As a chef, I can say this predicted recipe would not produce the dish shown in the image or described in the ground truth. The primary issue is the use of white beans instead of the green and wax beans depicted and specified. The herb mixture is also completely different from the shallots and basil in the ground truth. The predicted recipe includes an unnecessary oven preheating step and omits the crucial boiling step for the green/wax beans. While it technically generates a recipe for 'Herbed Bean Salad,' it's for a different type of salad entirely. The thinking proce